In [61]:
# Классификация описаний тендерных закупок
# Ремезов А.
# 
# Задача многоклассовой классификации;
# В коде предложены варианты создания признаков на основе текстовых данных
# (CountVectorizer, Tf-Idf, doc2vec) и обучения на них разных классификаторов.

# Полагаем, что ошибки на любых объектах равнозначны
# (исходя из данных о стоимости ручной разметки и ошибки классификатора);

# Соответственно, с помощью модели необходимо достичь точности выше 50% - 
# в таком случае машинная разметка будет окупать себя по сравнению с ручной.

import pandas as pd
import numpy as np
import datetime
import string
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn import utils

from pymystem3 import Mystem

import nltk
from nltk.corpus import stopwords

import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from tqdm import tqdm

In [2]:

t_path = 'C:/Users/Artem/Desktop/B2B-Center/'
path_train = t_path + 'train.csv'
path_test = t_path + 'test.csv'
result_path = t_path + 'result.csv'

rs = 42

In [62]:
nltk.download("stopwords")

In [4]:
# Загружаем датафреймы, изучаем данные
train_df = pd.read_csv(path_train, sep=',')
test_df = pd.read_csv(path_test, sep=',')

In [6]:
train_df.head(5)

,index,proc_name,target
0,0,право заключения договора на поставку насосов ...,40
1,1,инструмент для трубопрокатных станов хпт и хпт...,31
2,2,электрооборудование каталог hahsa flex,27
3,3,оказание услуг по восстановлению работоспособн...,60
4,4,"геодезические комплектующие и аксессуары,согла...",128


In [7]:
train_df.tail(5)

,index,proc_name,target
212085,212085,"фрезы отрезные, шпоночные гост 2679-93, гост 5...",31
212086,212086,техническое обслуживание систем пожаротушения ...,32
212087,212087,работы по разработке проектно-сметной документ...,18
212088,212088,приобретение двс мтз- ммз 245 - 1003015б1,21
212089,212089,поставка провода сип,27


In [8]:
test_df.head(5)

,index,proc_name
0,0,подшипник 2шсл 90\r\nподшипник 2007156\r\nподш...
1,1,система лазерной защиты fiessler\r\nподробное ...
2,2,запрос услуги: перевозка металлического сетчат...
3,3,поставка насосного агрегата grundfos sv 044 (d...
4,4,"определение подрядчика на выполнение смр, пнр ..."


In [9]:
train_df['target'].value_counts()

34     20831
27     10200
31      7675
60      7449
21      7385
17      7377
64      7281
69      5274
61      4711
28      4577
16      3866
39      3808
37      3761
25      3623
73      3295
5       3002
29      3001
40      2978
33      2771
48      2740
24      2650
32      2394
62      2313
26      2241
56      2218
46      2191
72      2068
20      1936
30      1936
68      1908
       ...  
98       275
126      261
95       258
144      253
63       241
139      240
94       237
10       211
45       205
128      201
130      190
123      159
88       158
78       132
104      126
7        116
141      109
66       106
1        106
142      103
107      102
109       97
8         97
108       89
2         78
50        70
4         69
116       49
127       29
11        20
Name: target, Length: 146, dtype: int64

In [10]:
# Проверка на NA
for col in train_df.columns:
    s = train_df[col].isnull()==True
    print('na values in ' + col + '...' + str(sum(s)))

na values in index...0
na values in proc_name...0
na values in target...0


In [11]:
# Очистка текста
def clean_text(text):
    # Приводим к нижнему регистру
    text = text.lower()

    # Заменяем переносы строки на пробелы
    text = re.sub("^\s+|\n|\r|\s+$", ' ', text)
    
    # Удаляем цифры
    text = re.sub(r'\d+', '', text)
    
    # Заменяем знаки препинания на пробелы (затем удалим их)
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Удаляем лишние пробелы
    text = text.strip()
    
    return text

In [ ]:
train_df['proc_name'] = train_df.apply(lambda x: clean_text(x['proc_name']), axis=1)
test_df['proc_name'] = test_df.apply(lambda x: clean_text(x['proc_name']), axis=1)

In [12]:
# Функция для лемматизации с использование Pymystem3

# Превращаем Series в список, собираем через разделитель,
# далее одну большую строку отдаем в лемматизатор,
# потом разбираем в столбец датафрейма обратно
# (для быстроты - по мотивам https://habr.com/ru/post/503420/ )

def lemmatize_column(df, col_name):
    print(datetime.datetime.now())

    m = Mystem()


    a = list(df[col_name])
    b = '<<>>'.join(a)
    c = m.lemmatize(b)
    d = ''.join(c)
    d = ' '.join(d.split())
    df[col_name + '_lemma'] = d.split('<<>>')

    df.drop([col_name], axis=1, inplace=True)

    print(datetime.datetime.now())
    
    return df

In [ ]:
train_df = lemmatize_column(train_df, 'proc_name')
test_df = lemmatize_column(test_df, 'proc_name')

In [14]:
# Заменяем NA после лемматизации (по одному значению получилось)
s = train_df['proc_name_lemma'].isnull()==True
train_df.loc[s, 'proc_name_lemma'] = 'unknown'

s = test_df['proc_name_lemma'].isnull()==True
test_df.loc[s, 'proc_name_lemma'] = 'unknown'

In [16]:
russian_stopwords = stopwords.words('russian')

In [17]:
train_df.tail(5)

,index,target,proc_name_lemma
212085,212085,31,фреза отрезной шпоночный гост гост сверло ц х ...
212086,212086,32,технический обслуживание система пожаротушение...
212087,212087,18,работа по разработка проектный сметный докумен...
212088,212088,21,приобретение двс мтз ммз б
212089,212089,27,поставка провод сип


In [18]:
# Стоп-слова из библиотеки Nltk
def remove_russian_stopwords(text):
    a = text.split(' ')
    a = [item for item in a if item not in russian_stopwords]
    return ' '.join(a)

train_df['proc_name_lemma'] = train_df.apply(lambda x: remove_russian_stopwords(x['proc_name_lemma']), axis=1)
test_df['proc_name_lemma'] = test_df.apply(lambda x: remove_russian_stopwords(x['proc_name_lemma']), axis=1)

In [19]:
train_df.tail(5)

,index,target,proc_name_lemma
212085,212085,31,фреза отрезной шпоночный гост гост сверло ц х ...
212086,212086,32,технический обслуживание система пожаротушение...
212087,212087,18,работа разработка проектный сметный документац...
212088,212088,21,приобретение двс мтз ммз б
212089,212089,27,поставка провод сип


In [40]:
# Первая модель - признаки на CountVectorizer + Tf-Idf,
# далее Байесовский классификатор

# Перемешиваем датафрейм
train_df = train_df.sample(n=len(train_df), random_state=rs)

In [41]:
X = train_df['proc_name_lemma']
y = train_df['target']

In [42]:
# Стратифицированная кросс-валидация
skf = StratifiedKFold(n_splits=5)

In [43]:
cnt = 0

for train_index, test_index in skf.split(X, y):
    cnt += 1
    

    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Устанавливаем параметр max_df, чтобы не использовать стоп-слова
    # и общие термины
    count_vect = CountVectorizer(max_df=0.2)
    X_train = count_vect.fit_transform(X_train)
    X_test = count_vect.transform(X_test)
    
    tfidf_transformer = TfidfTransformer()
    X_train = tfidf_transformer.fit_transform(X_train)
    X_test = tfidf_transformer.transform(X_test)
    
    NB_classifier = MultinomialNB().fit(X_train, y_train)
    predicted = NB_classifier.predict(X_test)
    print('(NB) Fold ' + str(cnt) + ', accuracy: ... ' + str(np.mean(predicted == y_test)))

(NB) Fold 1, accuracy: ... 0.5689598342515422
(NB) Fold 2, accuracy: ... 0.5728288016587343
(NB) Fold 3, accuracy: ... 0.5700209791858191
(NB) Fold 4, accuracy: ... 0.5710814381428706
(NB) Fold 5, accuracy: ... 0.5728819960814862


In [45]:
# Модель: признаки - doc2vec (используем Gensim), далее Логистическая регрессия

# Отложенная выборка
train, test = train_test_split(train_df, test_size=0.33, random_state=rs)

In [22]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

# Готовим данные, обучаем doc2vec

def get_doc_2_vec_data(train_df, test_df):
    
    train_tagged = train_df.apply(
        lambda x: TaggedDocument(words=x['proc_name_lemma'].split(), tags=[x['target']]), axis=1)
    
    # Функция используется также на неизвестных данных, в этом случае
    # в лейблы записываем 'index'
    if 'target' in test_df.columns:
        t_col = 'target'
    else:
        t_col = 'index'
    
    test_tagged = test_df.apply(
        lambda x: TaggedDocument(words=x['proc_name_lemma'].split(), tags=[x[t_col]]), axis=1)
    
    print(train_tagged.values[0])
    
    model_dbow = Doc2Vec(dm=0, vector_size=200, negative=5, hs=0, min_count=2, window=15)
    model_dbow.random.seed(rs)
    model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])
    
    for epoch in range(30):
        model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
        model_dbow.alpha -= 0.002
        model_dbow.min_alpha = model_dbow.alpha
        
    y_train, X_train = vec_for_learning(model_dbow, train_tagged)
    y_test, X_test = vec_for_learning(model_dbow, test_tagged)
    
    return y_train, X_train, y_test, X_test

In [46]:
y_train, X_train, y_test, X_test = get_doc_2_vec_data(train, test)

TaggedDocument(['международный', 'морской', 'доставка', 'груз', 'сборный', 'контейнер', 'проведение', 'маркетинг', 'доставка', 'груз', 'запасной', 'часть', 'печь', 'арендовать', 'сборный', 'контейнер', 'контейнер', 'перевозчик', 'маршрут', 'склад', 'поставщик', 'johannesburg', 'south', 'africa', 'морской', 'порт', 'г', 'владивосток', 'рф', 'склад', 'грузополучатель', 'г', 'амурск', 'рф'], [39])


100%|██████████████████████████████████████████████████████████████████████| 142100/142100 [00:00<00:00, 850577.34it/s]


In [52]:
# Логистическая регрессия на признаках doc2vec

multinomial_lr = LogisticRegression(multi_class='multinomial', solver='newton-cg', random_state=rs).fit(X_train, y_train)
predictions = multinomial_lr.predict(X_test)

In [60]:
print('Accuracy (Logistic Regression) - doc2vec features:')
print(str(np.mean(predictions==test['target'])))

Accuracy (Logistic Regression) on doc2vec features:
0.643820545792256


In [48]:
# Финальная модель на всех данных для обучения 
# (используем doc2vec + Логистическую регрессию)

train_df.sort_values(['index'], inplace=True)

In [23]:
y_train, X_train, y_test, X_test = get_doc_2_vec_data(train_df, test_df)

 38%|███████████████████████████▎                                           | 81412/212090 [00:00<00:00, 456558.13it/s]

TaggedDocument(['право', 'заключение', 'договор', 'поставка', 'насос', 'консольный', 'лот', 'право', 'заключение', 'договор', 'поставка', 'насос', 'консольный'], [40])


100%|██████████████████████████████████████████████████████████████████████| 212090/212090 [00:00<00:00, 671731.96it/s]


In [34]:
multinomial_lr = LogisticRegression(multi_class='multinomial', solver='newton-cg', random_state=rs).fit(X_train, y_train)
predictions = multinomial_lr.predict(X_test)

In [38]:
# Сохраняем датафрейм в необходимом формате (файл 'result.csv')

test_df['target'] = predictions

test_df.drop(['proc_name_lemma'], axis=1, inplace=True)

test_df.to_csv(result_path, sep=',', index=False)